In [ ]:
# 0. Install dependencies and verify GPU
!pip install transformers scikit-learn wandb sentencepiece -q

import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️ NO GPU DETECTED! Go to Runtime → Change runtime type → GPU')

In [ ]:
# 1. Mount Google Drive & Setup (RUN THIS FIRST!)
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Setup paths
DRIVE_FOLDER = '/content/drive/MyDrive/thesis_training'
os.makedirs(DRIVE_FOLDER, exist_ok=True)
os.makedirs(f'{DRIVE_FOLDER}/checkpoints', exist_ok=True)
os.makedirs('dataset', exist_ok=True)

# Copy dataset from Google Drive
DRIVE_DATASET = f'{DRIVE_FOLDER}/UNIFIED_ALL_SPLIT.csv'
LOCAL_DATASET = 'dataset/UNIFIED_ALL_SPLIT.csv'

if os.path.exists(DRIVE_DATASET):
    !cp '{DRIVE_DATASET}' '{LOCAL_DATASET}'
    print('✅ Dataset loaded from Google Drive!')
else:
    print(f'❌ Dataset not found! Please upload UNIFIED_ALL_SPLIT.csv to:')
    print(f'   Google Drive → My Drive → thesis_training/')
    print(f'   Then re-run this cell.')

# Checkpoint directory (saves to Drive - survives disconnects!)
CHECKPOINT_DIR = f'{DRIVE_FOLDER}/checkpoints/'
print(f'✅ Checkpoints will save to: {CHECKPOINT_DIR}')

!ls -la dataset/

In [ ]:
# 2. EDA: Sanity check splits and class balance
import pandas as pd

df = pd.read_csv('dataset/UNIFIED_ALL_SPLIT.csv')

print('Split counts:')
print(df['split'].value_counts(), '\n')

print('Per split language distribution:')
print(df.groupby('split')['language'].value_counts(), '\n')

print('hate_type distribution:')
print(df['hate_type'].value_counts(), '\n')

print('source_dataset distribution:')
print(df['source_dataset'].value_counts())

In [ ]:
# 3. HateDataset: PyTorch Dataset with tokenization and masking
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer

class HateDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=160):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = str(row['text'])
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        hate_type = int(row['hate_type'])
        target_group = int(row['target_group'])
        severity = int(row['severity'])
        
        hate_type_mask = hate_type != -1
        target_group_mask = target_group != -1
        severity_mask = severity != -1
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'hate_type': torch.tensor(max(0, hate_type), dtype=torch.long),
            'target_group': torch.tensor(max(0, target_group), dtype=torch.long),
            'severity': torch.tensor(max(0, severity), dtype=torch.long),
            'hate_type_mask': torch.tensor(hate_type_mask, dtype=torch.bool),
            'target_group_mask': torch.tensor(target_group_mask, dtype=torch.bool),
            'severity_mask': torch.tensor(severity_mask, dtype=torch.bool),
        }

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')
print('Tokenizer loaded.')

In [ ]:
# 4. MultiTaskXLMRRoberta Model
import torch.nn as nn
from transformers import XLMRobertaModel

class MultiTaskXLMRRoberta(nn.Module):
    def __init__(self, model_name='xlm-roberta-large', dropout=0.2,
                 n_hate_type=6, n_target_group=4, n_severity=4):
        super().__init__()
        self.backbone = XLMRobertaModel.from_pretrained(model_name)
        hidden_size = self.backbone.config.hidden_size
        
        self.dropout = nn.Dropout(dropout)
        self.hate_type_head = nn.Linear(hidden_size, n_hate_type)
        self.target_group_head = nn.Linear(hidden_size, n_target_group)
        self.severity_head = nn.Linear(hidden_size, n_severity)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        
        return (
            self.hate_type_head(cls_output),
            self.target_group_head(cls_output),
            self.severity_head(cls_output)
        )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# 5. Loss and evaluation functions WITH CLASS WEIGHTS
import torch.nn.functional as F
from sklearn.metrics import f1_score, classification_report
import numpy as np

def compute_class_weights(df, column, n_classes, smoothing=0.1):
    """Compute inverse frequency class weights with smoothing."""
    valid = df[df[column] != -1][column]
    counts = valid.value_counts().reindex(range(n_classes), fill_value=1).values
    weights = 1.0 / (counts + smoothing * len(valid))
    weights = weights / weights.sum() * n_classes
    return torch.tensor(weights, dtype=torch.float32)

def multitask_loss(hate_type_logits, target_group_logits, severity_logits,
                   targets, masks, task_weights=(1.0, 1.0, 1.0),
                   ht_class_weights=None, tg_class_weights=None, sv_class_weights=None):
    """Masked cross-entropy loss with optional class weights."""
    total_loss = 0.0
    n_tasks = 0
    
    ht_mask = masks['hate_type'].bool()
    if ht_mask.any():
        loss_ht = F.cross_entropy(hate_type_logits[ht_mask], targets['hate_type'][ht_mask], weight=ht_class_weights)
        total_loss += task_weights[0] * loss_ht
        n_tasks += 1
    
    tg_mask = masks['target_group'].bool()
    if tg_mask.any():
        loss_tg = F.cross_entropy(target_group_logits[tg_mask], targets['target_group'][tg_mask], weight=tg_class_weights)
        total_loss += task_weights[1] * loss_tg
        n_tasks += 1
    
    sv_mask = masks['severity'].bool()
    if sv_mask.any():
        loss_sv = F.cross_entropy(severity_logits[sv_mask], targets['severity'][sv_mask], weight=sv_class_weights)
        total_loss += task_weights[2] * loss_sv
        n_tasks += 1
    
    return total_loss / max(1, n_tasks)

def move_batch_to_device(batch):
    return {k: v.to(device) for k, v in batch.items()}

def evaluate(model, data_loader, task_weights=(1.0, 1.0, 1.0),
             ht_class_weights=None, tg_class_weights=None, sv_class_weights=None, verbose=False):
    model.eval()
    total_loss = 0.0
    n_batches = 0
    all_preds = {'hate_type': [], 'target_group': [], 'severity': []}
    all_labels = {'hate_type': [], 'target_group': [], 'severity': []}
    all_masks = {'hate_type': [], 'target_group': [], 'severity': []}
    
    with torch.no_grad():
        for batch in data_loader:
            batch = move_batch_to_device(batch)
            ht_logits, tg_logits, sv_logits = model(batch['input_ids'], batch['attention_mask'])
            targets = {k: batch[k] for k in ['hate_type', 'target_group', 'severity']}
            masks = {k: batch[f'{k}_mask'] for k in targets.keys()}
            loss = multitask_loss(ht_logits, tg_logits, sv_logits, targets, masks, task_weights,
                                  ht_class_weights, tg_class_weights, sv_class_weights)
            total_loss += loss.item()
            n_batches += 1
            all_preds['hate_type'].extend(ht_logits.argmax(dim=1).cpu().numpy())
            all_preds['target_group'].extend(tg_logits.argmax(dim=1).cpu().numpy())
            all_preds['severity'].extend(sv_logits.argmax(dim=1).cpu().numpy())
            for task in ['hate_type', 'target_group', 'severity']:
                all_labels[task].extend(targets[task].cpu().numpy())
                all_masks[task].extend(masks[task].cpu().numpy())
    
    metrics = {'loss': total_loss / max(1, n_batches)}
    for task in ['hate_type', 'target_group', 'severity']:
        mask = np.array(all_masks[task]).astype(bool)
        if mask.sum() > 0:
            preds = np.array(all_preds[task])[mask]
            labels = np.array(all_labels[task])[mask]
            metrics[f'{task}_macro_f1'] = f1_score(labels, preds, average='macro', zero_division=0)
            metrics[f'{task}_micro_f1'] = f1_score(labels, preds, average='micro', zero_division=0)
            if verbose:
                print(f'\n{task.upper()} Classification Report:')
                print(classification_report(labels, preds, zero_division=0))
        else:
            metrics[f'{task}_macro_f1'] = None
            metrics[f'{task}_micro_f1'] = None
    return metrics

print('✅ Loss and evaluation functions defined with CLASS WEIGHTS support.')

In [ ]:
# 6. Setup data loaders + COMPUTE CLASS WEIGHTS
import os

SEED = 1337
MAX_LENGTH = 160
BATCH_SIZE = 16

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

train_df = df[df['split'] == 'train'].reset_index(drop=True)
val_df = df[df['split'] == 'val'].reset_index(drop=True)
test_df = df[df['split'] == 'test'].reset_index(drop=True)
print(f'Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}')

# Compute class weights from training data (helps with imbalanced classes!)
ht_weights = compute_class_weights(train_df, 'hate_type', 6).to(device)
tg_weights = compute_class_weights(train_df, 'target_group', 4).to(device)
sv_weights = compute_class_weights(train_df, 'severity', 4).to(device)

print(f'\n📊 Class Weights (higher = more focus on that class):')
print(f'  hate_type:    {[f"{w:.2f}" for w in ht_weights.tolist()]}')
print(f'  target_group: {[f"{w:.2f}" for w in tg_weights.tolist()]}')
print(f'  severity:     {[f"{w:.2f}" for w in sv_weights.tolist()]}')

train_dataset = HateDataset(train_df, tokenizer, max_length=MAX_LENGTH)
val_dataset = HateDataset(val_df, tokenizer, max_length=MAX_LENGTH)
test_dataset = HateDataset(test_df, tokenizer, max_length=MAX_LENGTH)

# Use num_workers=2 for faster data loading on Colab
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
print(f'\nData loaders created. Batches - Train: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)}')

In [ ]:
# 7. Training function with CLASS WEIGHTS + macro F1 early stopping
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import time

training_config = {
    'epochs': 5,
    'learning_rate': 1e-5,    # Lower LR for stability
    'weight_decay': 1e-2,
    'warmup_ratio': 0.1,
    'grad_clip': 1.0,
    'patience': 3,
    'dropout': 0.3,           # Slightly higher dropout
    'task_weights': (1.0, 1.0, 1.0),
    'use_class_weights': True
}

def train_model(train_loader, val_loader, config=None, run_name='xlmr_run', use_wandb=False, resume_from=None,
                ht_class_weights=None, tg_class_weights=None, sv_class_weights=None):
    if config is None: config = training_config
    if use_wandb:
        import wandb
        wandb.init(project='multilingual-hate-detection', name=run_name, config=config, resume='allow')
    
    model = MultiTaskXLMRRoberta(dropout=config['dropout']).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    total_steps = len(train_loader) * config['epochs']
    warmup_steps = int(total_steps * config['warmup_ratio'])
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    
    best_val_loss = float('inf')
    best_macro_f1 = 0.0
    patience_counter = 0
    start_epoch = 1
    history = []
    
    best_ckpt_path = os.path.join(CHECKPOINT_DIR, f'{run_name}_best.pt')
    
    if resume_from and os.path.exists(resume_from):
        print(f'Resuming from checkpoint: {resume_from}')
        checkpoint = torch.load(resume_from, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_loss = checkpoint.get('best_val_loss', float('inf'))
        best_macro_f1 = checkpoint.get('best_macro_f1', 0.0)
        patience_counter = checkpoint.get('patience_counter', 0)
        history = checkpoint.get('history', [])
        print(f'Resumed from epoch {checkpoint["epoch"]}. Starting epoch {start_epoch}.')
    
    for epoch in range(start_epoch, config['epochs'] + 1):
        model.train()
        start = time.time()
        running_loss = 0.0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch}/{config["epochs"]}', leave=True)
        for batch_idx, batch in enumerate(pbar):
            batch = move_batch_to_device(batch)
            optimizer.zero_grad()
            logits = model(batch['input_ids'], batch['attention_mask'])
            targets = {k: batch[k] for k in ['hate_type', 'target_group', 'severity']}
            masks = {k: batch[f'{k}_mask'] for k in targets.keys()}
            loss = multitask_loss(*logits, targets, masks, config['task_weights'],
                                  ht_class_weights, tg_class_weights, sv_class_weights)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), config['grad_clip'])
            optimizer.step()
            scheduler.step()
            running_loss += loss.item()
            avg_loss = running_loss / (batch_idx + 1)
            pbar.set_postfix({'loss': f'{avg_loss:.4f}'})
        
        train_loss = running_loss / max(1, len(train_loader))
        
        print(f'Evaluating on validation set...')
        val_metrics = evaluate(model, val_loader, config['task_weights'],
                               ht_class_weights, tg_class_weights, sv_class_weights)
        val_loss = val_metrics['loss']
        
        # Compute average macro F1 across tasks
        macro_f1s = [val_metrics.get(f'{t}_macro_f1', 0) or 0 for t in ['hate_type', 'target_group', 'severity']]
        avg_macro_f1 = sum(macro_f1s) / len(macro_f1s)
        
        epoch_time = time.time() - start
        log_payload = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss,
                       'avg_macro_f1': avg_macro_f1, 'epoch_time': epoch_time, **val_metrics}
        history.append(log_payload)
        if use_wandb: wandb.log(log_payload)
        
        print(f'Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, avg_macro_f1={avg_macro_f1:.4f}, time={epoch_time:.1f}s')
        print(f'  hate_type_macro_f1={val_metrics.get("hate_type_macro_f1", 0):.4f}, target_group_macro_f1={val_metrics.get("target_group_macro_f1", 0):.4f}, severity_macro_f1={val_metrics.get("severity_macro_f1", 0):.4f}')
        
        # Save epoch checkpoint
        epoch_ckpt_path = os.path.join(CHECKPOINT_DIR, f'{run_name}_epoch{epoch}.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_val_loss': best_val_loss,
            'best_macro_f1': best_macro_f1,
            'patience_counter': patience_counter,
            'history': history,
            'config': config
        }, epoch_ckpt_path)
        print(f'  💾 Epoch checkpoint saved to {epoch_ckpt_path}')
        
        # Save best model based on MACRO F1 (better for imbalanced data)
        if avg_macro_f1 > best_macro_f1:
            best_macro_f1 = avg_macro_f1
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_ckpt_path)
            print(f'  ✓ New best checkpoint saved! (avg_macro_f1={avg_macro_f1:.4f})')
        else:
            patience_counter += 1
            print(f'  No improvement. Patience: {patience_counter}/{config["patience"]}')
            if patience_counter >= config['patience']:
                print('Early stopping triggered.')
                break
    
    if use_wandb: wandb.finish()
    return best_ckpt_path, history

print('✅ Training function defined with CLASS WEIGHTS + macro F1 early stopping.')

In [ ]:
# 8. (Optional) W&B Login - uncomment if you want experiment tracking
# import wandb
# wandb.login(key='YOUR_WANDB_KEY')  # Or just run wandb.login() to authenticate interactively

In [ ]:
# 9. 🚀 FULL TRAINING WITH CLASS WEIGHTS - Run this cell!
print(f'🚀 Starting IMPROVED training on {len(train_dataset)} samples...')
print(f'Device: {device}')
print(f'📁 Checkpoints saving to: {CHECKPOINT_DIR}')
print(f'\n📊 Using class weights to handle imbalance!')
print(f'  hate_type weights:    {[f"{w:.2f}" for w in ht_weights.tolist()]}')
print(f'  target_group weights: {[f"{w:.2f}" for w in tg_weights.tolist()]}')
print(f'  severity weights:     {[f"{w:.2f}" for w in sv_weights.tolist()]}')
print('=' * 50)

# To resume from a previous epoch, set this:
# resume_checkpoint = '/content/drive/MyDrive/thesis_training/checkpoints/xlmr_v2_epoch2.pt'
resume_checkpoint = None

best_checkpoint_full, history_full = train_model(
    train_loader, val_loader, 
    config=training_config,
    run_name='xlmr_v2_classweights',
    use_wandb=False,
    resume_from=resume_checkpoint,
    ht_class_weights=ht_weights,
    tg_class_weights=tg_weights,
    sv_class_weights=sv_weights
)
print('\n✅ Training complete!')
print(f'📁 Best checkpoint: {best_checkpoint_full}')
print('History:', history_full)

In [ ]:
# 10. Final Evaluation with detailed per-class metrics
print('Loading best checkpoint and evaluating...')
best_model_full = MultiTaskXLMRRoberta().to(device)
best_model_full.load_state_dict(torch.load(best_checkpoint_full, map_location=device))

print('\n' + '='*60)
print('=== VALIDATION SET RESULTS ===')
print('='*60)
val_results_full = evaluate(best_model_full, val_loader, verbose=True,
                            ht_class_weights=ht_weights, tg_class_weights=tg_weights, sv_class_weights=sv_weights)
print('\nSummary:')
for k, v in val_results_full.items():
    if v is not None: print(f'  {k}: {v:.4f}')

print('\n' + '='*60)
print('=== TEST SET RESULTS ===')
print('='*60)
test_results_full = evaluate(best_model_full, test_loader, verbose=True,
                             ht_class_weights=ht_weights, tg_class_weights=tg_weights, sv_class_weights=sv_weights)
print('\nSummary:')
for k, v in test_results_full.items():
    if v is not None: print(f'  {k}: {v:.4f}')

print(f'\n📁 Best checkpoint: {best_checkpoint_full}')

In [ ]:
# 10.5 🔮 Inference: Predict on new text
# CORRECTED LABELS based on main.py mapping!
HATE_TYPE_LABELS = {
    0: 'not_hate/other',
    1: 'political',
    2: 'religious',
    3: 'gender',
    4: 'personal_attack',
    5: 'geopolitical'
}
TARGET_GROUP_LABELS = {0: 'other/none', 1: 'individual', 2: 'organization/group', 3: 'community'}
SEVERITY_LABELS = {0: 'none', 1: 'low', 2: 'medium', 3: 'high'}

def predict(text, model=None, return_probs=False):
    """Predict hate type, target group, and severity for a given text."""
    if model is None:
        model = best_model_full
    
    model.eval()
    encoding = tokenizer(text, max_length=160, padding='max_length', truncation=True, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        ht_logits, tg_logits, sv_logits = model(input_ids, attention_mask)
    
    ht_pred = ht_logits.argmax(dim=1).item()
    tg_pred = tg_logits.argmax(dim=1).item()
    sv_pred = sv_logits.argmax(dim=1).item()
    
    result = {
        'text': text[:100] + '...' if len(text) > 100 else text,
        'hate_type': HATE_TYPE_LABELS[ht_pred],
        'target_group': TARGET_GROUP_LABELS[tg_pred],
        'severity': SEVERITY_LABELS[sv_pred],
    }
    
    if return_probs:
        result['hate_type_probs'] = torch.softmax(ht_logits, dim=1).cpu().numpy()[0]
        result['target_group_probs'] = torch.softmax(tg_logits, dim=1).cpu().numpy()[0]
        result['severity_probs'] = torch.softmax(sv_logits, dim=1).cpu().numpy()[0]
    
    return result

# Test examples
print('🔮 Testing predictions...\n')
test_texts = [
    "তুই একটা বোকা",           # Bengali - personal attack
    "You're such an idiot",   # English - personal attack  
    "tui ekta pagol",         # Banglish - personal attack
    "Have a nice day!",       # English - not hate
]

for text in test_texts:
    result = predict(text)
    print(f"Text: {result['text']}")
    print(f"  → Hate: {result['hate_type']}, Target: {result['target_group']}, Severity: {result['severity']}\n")

In [ ]:
# 11. Download the trained model from Google Drive
from google.colab import files
import shutil

# Checkpoints are already in Google Drive! Just download them.
DRIVE_CHECKPOINT_DIR = '/content/drive/MyDrive/thesis_training/checkpoints'

print('📁 Checkpoints saved in Google Drive:')
!ls -la {DRIVE_CHECKPOINT_DIR}

# Create a zip for download
shutil.make_archive('trained_model', 'zip', DRIVE_CHECKPOINT_DIR)
print('\n📥 Downloading trained_model.zip...')
files.download('trained_model.zip')

print('''
✅ Download complete!

Your checkpoints are ALSO permanently saved in Google Drive at:
  My Drive/thesis_training/checkpoints/

You can access them anytime, even after runtime ends!
''')